# Compare zero-crossings to auto-detected sniff inhalation onsets
### Get zero crossings

In [44]:
import zerocrossing
rootpath = "./data/190603/"
files = ["1953_1_04_D2019_6_3T12_29_13_odor.h5"]
# files = ["1953_1_05_D2019_6_3T12_47_22_odor.h5"]
crossings = zerocrossing.getCrossingsFromFiles(files, rootpath)
# Only one file, so let's get rid of the dict
crossings = crossings[files[0]]

In [45]:
print(f"len(crossings) = {len(crossings)}")
[[len(x) for x in crossing[:5]] for crossing in crossings] 

len(crossings) = 45


[[245, 85, 36],
 [85, 39, 35, 179, 84],
 [61, 38, 36, 53, 46],
 [69, 37, 17, 56, 103],
 [105, 42, 37, 89, 45],
 [67, 36, 16, 57, 47],
 [69, 38, 36, 57, 28],
 [70, 38, 36, 60, 47],
 [69, 37, 17, 60, 48],
 [69, 37, 37, 61, 47],
 [73, 37, 17, 65, 45],
 [69, 35, 15, 57, 48],
 [67, 36, 16, 60, 48],
 [69, 37, 17, 56, 30],
 [70, 38, 35, 61, 48],
 [89, 37, 34, 60, 47],
 [141, 90, 36, 88, 48],
 [67, 36, 36, 61, 48],
 [72, 35, 16, 56, 50],
 [69, 38, 36, 57, 48],
 [68, 37, 17, 56, 50],
 [70, 38, 35, 66, 44],
 [71, 38, 35, 60, 48],
 [69, 38, 16, 60, 48],
 [69, 37, 17, 60, 48],
 [72, 34, 17, 60, 49],
 [72, 39, 34, 60, 50],
 [69, 38, 36, 64, 47],
 [71, 38, 34, 57, 29],
 [71, 38, 35, 66, 85],
 [68, 35, 16, 65, 45],
 [71, 35, 17, 138, 156],
 [97, 38, 36, 57, 28],
 [139, 88, 36, 60, 47],
 [72, 35, 17, 60, 48],
 [73, 38, 17, 56, 30],
 [70, 37, 36, 66, 44],
 [69, 37, 17, 57, 27],
 [135, 85, 35, 66, 44],
 [68, 36, 16, 60, 49],
 [78, 38, 36, 126, 48],
 [71, 37, 35, 61, 49],
 [72, 35, 36, 60, 47],
 [71, 38,

In [46]:
# Crossing positions in first sniff sampling 
print(crossings[0][0].nonzero())
print(len(crossings[0][0]))

(array([ 22, 121, 163]),)
245


### Get Events data to assign timestamp to zero crossings

In [47]:
import h5py
crossingTimestamps = []
with h5py.File(rootpath + files[0], "r") as h5File:
    numFrameTriggers = sum([sum([len(x) for x in h5File["Trial{:04d}".format(i_trial)]["frame_triggers"]]) for i_trial in range(1,46)])
    print(f"Total frame triggers: {numFrameTriggers}")
    for i_trial, crossing in enumerate(crossings):
#     for i_trial in range(1, len(h5File)):
        events = h5File["Trial{:04d}".format(i_trial+1)]["Events"]
        timestamps = []
        for i_sample, sample in enumerate(crossing):
            crossingIndexes = sample.nonzero()[0]
            # Need negative offsets -- matching Events packet_sent_time is ending timestamp
            offsets = crossingIndexes - len(sample)
            timestamps.append(events["packet_sent_time"][i_sample] + offsets)
            if i_sample > 0:
                # confirm 1kHz sample-rate
                assert (len(sample) + events["packet_sent_time"][i_sample - 1]) == events["packet_sent_time"][i_sample]
        crossingTimestamps.append(timestamps)


Total frame triggers: 11679


In [48]:
print(crossingTimestamps[0])

[array([2223056, 2223155, 2223197]), array([2223306, 2223352, 2223363]), array([2223367])]


### Compare crossings to auto-detected inhalation onset timestamps

In [49]:
exactMatches = []
with h5py.File(rootpath + files[0], "r") as h5File:
    trials = h5File['Trials']
    inhalationOnsets = trials['inh_onset']
    for i_trial, onsetTimestamp in enumerate(inhalationOnsets):
        curCrossings = crossingTimestamps[i_trial]
        exactMatches.append(any([onsetTimestamp in eventCrossings for eventCrossings in curCrossings]))
print(f"Any matches? {any(exactMatches)}")

Any matches? True


In [56]:
import numpy as np
print(f"{sum(exactMatches)} matches out of {len(exactMatches)} at trials {np.nonzero(exactMatches)[0]}.")

2 matches out of 45 at trials [36 39].


### Get frames corresponding to each timestamp